1) Return specific pieces of information (rather than just downloading a whole page)
2) Iterate over multiple pages/queries
3) Save the data to your computer

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import json
import requests
from scrapy.linkextractors import LinkExtractor

#Build a crawler to crawl Google's top news articles and pull articles containing 'bitcoin'
class GoogleSpider(scrapy.Spider):
    name = "GS"
    
    allowed_domains = ['newsapi.org']
    
    # Here is where we insert our API call to get Google news articles related to bitcoin.
    start_urls = ['https://newsapi.org/v2/everything?q=bitcoin&apiKey=2613ce5e838a464b814b7d5b4c2e6bf8'
                 ,'https://newsapi.org/v2/everything?q=ethereum&apiKey=2613ce5e838a464b814b7d5b4c2e6bf8']
      
    # Identifying the information we want from the query response and extracting it using xpath.
    def parse(self, response):
        data = json.loads(response.body_as_unicode())
        data2 = []
        for article in data['articles']:
            yield {
                'url' : article['url']
            }
                

process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'GoogleLinks5.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'MatthewGoogleNewsCrawler (makennedy626@gmail.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    # We use CLOSESPIDER_PAGECOUNT to limit our scraper to the first 10 links.    
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

# Starting the crawler with our spider.
process.crawl(GoogleSpider)
process.start()
print('First 100 links extracted!')

First 100 links extracted!
